In [1]:
from utils.util import parse_args, get_reader, load_model, get_trainer, get_out_filename, write_eval_performance, get_tagset
import time
from torch.utils.data import DataLoader
from utils.util import wnut_iob
import os

In [27]:
checkpoint = "./roberta-train/lightning_logs/version_0/checkpoints/epoch=4-step=4784.ckpt"
checkpoint = "./roberta-train/lightning_logs/version_0/checkpoints/roberta-train_timestamp_1638429490.282762_final.ckpt"
#checkpoint = "./roberta-finetune/lightning_logs/version_0/checkpoints/"
checkpoint = "roberta-finetune/lightning_logs/version_1/checkpoints/finetune_timestamp_1638516609.385282_final.ckpt"
train_file = "./training_data/EN-English/en_train.conll"
dev_file = "./training_data/EN-English/en_dev.conll"
output_dir = "roberta-evaluate"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
iob_tagging = wnut_iob

In [28]:
test_data = get_reader(file_path=dev_file, target_vocab=wnut_iob, max_instances=-1, max_length=50, encoder_model="roberta-base")
model = load_model(checkpoint, tag_to_id=wnut_iob)
trainer = get_trainer(is_test=True)
out = trainer.test(model, test_dataloaders=DataLoader(test_data, batch_size=8, collate_fn=model.collate_batch))
eval_file = get_out_filename(output_dir, checkpoint, prefix="eval")
write_eval_performance(out, eval_file)

2021-12-03 16:12:47 - INFO - reader - Reading file ./training_data/EN-English/en_dev.conll
2021-12-03 16:12:48 - INFO - reader - Finished reading 800 instances from file ./training_data/EN-English/en_dev.conll
Some weights of the model checkpoint at ./roberta-retrained/checkpoint-5000 were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ./roberta-r

Testing: 100%|██████████| 100/100 [01:02<00:00,  1.54it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ALLPRED': 1241.0,
 'ALLRECALLED': 1042.0,
 'ALLTRUE': 1230.0,
 'F1@CORP': 0.7900552749633789,
 'F1@CW': 0.6277777552604675,
 'F1@GRP': 0.7823834419250488,
 'F1@LOC': 0.8506224155426025,
 'F1@PER': 0.9103214740753174,
 'F1@PROD': 0.6206896305084229,
 'MD@F1': 0.843383252620697,
 'MD@P': 0.8396454453468323,
 'MD@R': 0.8471544981002808,
 'P@CORP': 0.8461538553237915,
 'P@CW': 0.614130437374115,
 'P@GRP': 0.7704081535339355,
 'P@LOC': 0.8266128897666931,
 'P@PER': 0.8936877250671387,
 'P@PROD': 0.6293706297874451,
 'R@CORP': 0.7409326434135437,
 'R@CW': 0.6420454382896423,
 'R@GRP': 0.7947368621826172,
 'R@LOC': 0.8760683536529541,
 'R@PER': 0.9275861978530884,
 'R@PROD': 0.6122449040412903,
 'loss': 5.672953128814697,
 'micro@F1': 0.7859166264533997,
 'micro@P': 0.7824335098266602,
 'micro@R': 0.7894309163093567}
-----------

2021-12-03 16:13:52 - INFO - util - Finished writing evaluation performance for roberta-evaluate/eval_base_finetune_timestamp_1638516609.385282_final.tsv


In [29]:
batch_size = 8
test_dataloader = DataLoader(test_data, batch_size=batch_size, collate_fn=model.collate_batch)
output_result_file = os.path.join(output_dir, "details_pred.csv")
f = open(output_result_file, "w")
for batch in test_dataloader:
    output = model.perform_forward_step(batch)
    pred_result = output["pred_results"]
    for i in range(batch_size):
        input_ids = batch[0][i]
        pred_token_tag = pred_result[i]
        metadata_token_tag = batch[3][i]
        for (start_pos, end_pos), (pred_stat_pos, pred_end_pos) in zip(metadata_token_tag, pred_token_tag):
            sub_tokens = test_data.tokenizer.convert_ids_to_tokens(input_ids[start_pos: end_pos+1])
            pred_sub_tokens = test_data.tokenizer.convert_ids_to_tokens(input_ids[start_pos: end_pos+1])
            tag = metadata_token_tag[(start_pos, end_pos)]
            pred_tag = pred_token_tag[(pred_stat_pos, pred_end_pos)]
            for sub_token1, sub_token2 in zip(sub_tokens, pred_sub_tokens):
                f.write("{}{}{}{}{}{}{}".format(sub_token1, ",", tag, ",", sub_token2, ",", pred_tag))
                f.write("\n")
                
f.close() 

TypeError: write() takes exactly one argument (7 given)

In [37]:
test_data.tokenizer("among others , he was chairman of the standing conference of regional sports federations and member of the südwestrundfunk broadcasting council")

{'input_ids': [0, 31636, 643, 2156, 37, 21, 2243, 9, 5, 2934, 1019, 9, 2174, 1612, 32584, 1635, 8, 919, 9, 5, 579, 2768, 417, 10823, 2070, 1187, 506, 6435, 16687, 1676, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
test_data.tokenizer.convert_ids_to_tokens([1])

['<pad>',
 'e',
 'ase',
 'on',
 'down',
 'the',
 'road',
 'âĢĶ',
 'ch',
 'arl',
 'ie',
 'sm',
 'alls',
 '(',
 'd',
 'iana',
 'ross',
 'and',
 'm',
 'ichael',
 'jack',
 'son',
 'in',
 'the',
 'w',
 'iz',
 ')',
 '<pad>']

In [31]:
print(test_data.tokenizer.convert_ids_to_tokens([0]))

['<s>']
